# Training the LSTM Model
Running this notebook preprocesses the midi files and passes it to the neural network for training

### Imports

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


Change the values below in order to tweak the model's behavior

In [17]:
# Config
memory_length = 16 # use 16, 32, 64, etc.
num_epochs = 100

The function 'get_notes' retrieves every note and chord that occurs in all the midi files in the '/midi' directory

In [18]:
def get_notes():
    notes = []
    for file in glob.glob("data/midi/*.mid"):
        midi = converter.parse(file)
        print("Parsing %s" % file)
        notes_to_parse = None
        
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    return notes

The function 'prepare_sequences' preprocesses the data input 

In [19]:
def prepare_sequences(notes, n_vocab, memory_len):
    # get all pitch names
    pitchnames = sorted(set(item for item in notes))
    
     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    network_input = []
    network_output = []
    
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - memory_len, 1):
        sequence_in = notes[i:i + memory_len]
        sequence_out = notes[i + memory_len]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, memory_len, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [20]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]),
                   recurrent_dropout=0.3, return_sequences=True))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [21]:
def train(model, network_input, network_output, num_epochs):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0,
                                 save_best_only=True, mode='min')
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=num_epochs, batch_size=128, callbacks=callbacks_list)

The below function trains the Neural Network to generate music

In [22]:
def train_network(memory_len, num_epochs):
    notes = get_notes()
    n_vocab = len(set(notes)) # get amount of pitch names
    network_input, network_output = prepare_sequences(notes, n_vocab, memory_len)
    model = create_network(network_input, n_vocab)
    train(model, network_input, network_output, num_epochs)

#### Run This to Train the Model

In [23]:
train_network(memory_length, num_epochs)

Parsing data/midi\0fithos.mid
Parsing data/midi\8.mid
Parsing data/midi\ahead_on_our_way_piano.mid
Parsing data/midi\AT.mid
Parsing data/midi\balamb.mid
Parsing data/midi\bcm.mid
Parsing data/midi\BlueStone_LastDungeon.mid
Parsing data/midi\braska.mid
Parsing data/midi\caitsith.mid
Parsing data/midi\Cids.mid
Parsing data/midi\cosmo.mid
Parsing data/midi\costadsol.mid
Parsing data/midi\dayafter.mid
Parsing data/midi\decisive.mid
Parsing data/midi\dontbeafraid.mid
Parsing data/midi\DOS.mid
Parsing data/midi\electric_de_chocobo.mid
Parsing data/midi\Eternal_Harvest.mid
Parsing data/midi\EyesOnMePiano.mid
Parsing data/midi\ff11_awakening_piano.mid
Parsing data/midi\ff1battp.mid
Parsing data/midi\FF3_Battle_(Piano).mid
Parsing data/midi\FF3_Third_Phase_Final_(Piano).mid
Parsing data/midi\ff4-airship.mid
Parsing data/midi\Ff4-BattleLust.mid
Parsing data/midi\ff4-fight1.mid
Parsing data/midi\ff4-town.mid
Parsing data/midi\FF4.mid
Parsing data/midi\ff4pclov.mid
Parsing data/midi\ff4_piano_coll

KeyboardInterrupt: 

## Visualizing the Network

In [25]:
from keras_sequential_ascii import keras2ascii

notes = get_notes()
n_vocab = len(set(notes)) # get amount of pitch names
network_input, network_output = prepare_sequences(notes, n_vocab, memory_length)
model = create_network(network_input, n_vocab)

keras2ascii(model)
    
tf.keras.utils.plot_model(model,to_file="model.png")

Parsing data/midi\0fithos.mid
Parsing data/midi\8.mid
Parsing data/midi\ahead_on_our_way_piano.mid
Parsing data/midi\AT.mid
Parsing data/midi\balamb.mid
Parsing data/midi\bcm.mid
Parsing data/midi\BlueStone_LastDungeon.mid
Parsing data/midi\braska.mid
Parsing data/midi\caitsith.mid
Parsing data/midi\Cids.mid
Parsing data/midi\cosmo.mid
Parsing data/midi\costadsol.mid
Parsing data/midi\dayafter.mid
Parsing data/midi\decisive.mid
Parsing data/midi\dontbeafraid.mid
Parsing data/midi\DOS.mid
Parsing data/midi\electric_de_chocobo.mid
Parsing data/midi\Eternal_Harvest.mid
Parsing data/midi\EyesOnMePiano.mid
Parsing data/midi\ff11_awakening_piano.mid
Parsing data/midi\ff1battp.mid
Parsing data/midi\FF3_Battle_(Piano).mid
Parsing data/midi\FF3_Third_Phase_Final_(Piano).mid
Parsing data/midi\ff4-airship.mid
Parsing data/midi\Ff4-BattleLust.mid
Parsing data/midi\ff4-fight1.mid
Parsing data/midi\ff4-town.mid
Parsing data/midi\FF4.mid
Parsing data/midi\ff4pclov.mid
Parsing data/midi\ff4_piano_coll